In [3]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

# Fine-Tuning Amazon Nova models with Bedrock using API


In [ ]:
! pip install -r ../requirements.txt

### install the required packages

In [2]:
import boto3
from botocore.config import Config
import logging
from enum import Enum
import boto3
import json
from datetime import datetime
import time


### Connecting to Bedrock and list fine-tunable models 

In [ ]:
my_config = Config(
    region_name = 'us-east-1',
    retries = {
        'max_attempts': 5,
        'mode': 'standard'
    }
)

bedrock = boto3.client(service_name="bedrock", config=my_config)
bedrock_runtime = boto3.client(service_name="bedrock-runtime", config=my_config)


for model in bedrock.list_foundation_models(
    byCustomizationType="FINE_TUNING")["modelSummaries"]:
    for key, value in model.items():
        print(key, ":", value)
    print("-----\n")

### Get the data location as S3 URI

In [ ]:
iam = boto3.client("iam")
YOUR_BUCKET_NAME= "" #put the s3 bucket name which has the data for FT

train_s3_uri = f"s3://{YOUR_BUCKET_NAME}/formatted_train_ft.jsonl"
val_s3_uri = f"s3://{YOUR_BUCKET_NAME}/formatted_test_ft.jsonl"
output_s3_uri = f"s3://{YOUR_BUCKET_NAME}/output/"


### Use/Create IAM roles (uncomment when doing the first time)

In [ ]:
'''
ROLE_NAME ="" #The IAM role that grants permissions to interact with Bedrock services.
POLICY_NAME="" #The IAM policy that defines the permissions for the role or user.
ACCNT_NUMBER="" #The AWS account ID associated with the Bedrock service usage.

iam.create_role(
    RoleName=f"{ROLE_NAME}", 
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ] 
    })
)
iam.create_policy(
    PolicyName=f"{POLICY_NAME}",
    PolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:*",                    
                ],
                "Resource": [
                    "arn:aws:s3:::tooluse-olympus-new-east1",
                    "arn:aws:s3:::tooluse-olympus-new-east1/*"
                    
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:PrincipalAccount": f"{ACCNT_NUMBER}"
                    }
                }
            }
        ]
    }
    )
)
iam.attach_role_policy(
    RoleName= ROLE_NAME,
    PolicyArn=f"arn:aws:iam::{ACCNT_NUMBER}:policy/{POLICY_NAME}"
)
'''

In [ ]:
role_arn = f"arn:aws:iam::{ACCNT_NUMBER}:role/{POLICY_NAME}"

## 2. Fine-tuning

Now we can create the fine-tuning job

In [ ]:
ts = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
job_name = f"tooluse-nova-{ts}"
model_name = f"tooluse-nova-{ts}"


In [ ]:
#hyperparameters
epochs = "1"
batchsize = "1"
lr = "1.00E-07"
warmup = "5"

In [ ]:
training_job_response = bedrock.create_model_customization_job(
    jobName=job_name,
    customModelName=model_name,
    roleArn=role_arn,
    baseModelIdentifier="arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0"", # change accordingly
    trainingDataConfig={"s3Uri": train_s3_uri},
    validationDataConfig={"validators": [{
        "s3Uri": val_s3_uri
    }]},
    outputDataConfig={"s3Uri": output_s3_uri},
    hyperParameters={'epochCount': epochs, 'batchSize': batchsize, 'learningRate': lr, 'learningRateWarmupSteps': warmup}
)

In [ ]:
jobArn = training_job_response.get('jobArn')
jobArn

Let's periodically check in on the progress. The trainig job's duration varies by epochs, batch size, and data size.

In [ ]:
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)